# LightGBM

[LightGBM](https://github.com/Microsoft/LightGBM) is an open-source,
distributed, high-performance gradient boosting (GBDT, GBRT, GBM, or
MART) framework. This framework specializes in creating high-quality and
GPU enabled decision tree algorithms for ranking, classification, and
many other machine learning tasks. LightGBM is part of Microsoft's
[DMTK](http://github.com/microsoft/dmtk) project.

### Advantages of LightGBM

-   **Composability**: LightGBM models can be incorporated into existing
    SparkML Pipelines, and used for batch, streaming, and serving
    workloads.
-   **Performance**: LightGBM on Spark is 10-30% faster than SparkML on
    the Higgs dataset, and achieves a 15% increase in AUC.  [Parallel
    experiments](https://github.com/Microsoft/LightGBM/blob/master/docs/Experiments.rst#parallel-experiment)
    have verified that LightGBM can achieve a linear speed-up by using
    multiple machines for training in specific settings.
-   **Functionality**: LightGBM offers a wide array of [tunable
    parameters](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst),
    that one can use to customize their decision tree system. LightGBM on
    Spark also supports new types of problems such as quantile regression.
-   **Cross platform** LightGBM on Spark is available on Spark, PySpark, and SparklyR

### LightGBM Usage:

- LightGBMClassifier: used for building classification models. For example, to predict whether a company will bankrupt or not, we could build a binary classification model with LightGBMClassifier.
- LightGBMRegressor: used for building regression models. For example, to predict the house price, we could build a regression model with LightGBMRegressor.
- LightGBMRanker: used for building ranking models. For example, to predict website searching result relevance, we could build a ranking model with LightGBMRanker.

## Bankruptcy Prediction with LightGBM Classifier

<img src="https://mmlspark.blob.core.windows.net/graphics/Documentation/bankruptcy image.png" width="800" style="float: center;"/>

In this example, we use LightGBM to build a classification model in order to predict bankruptcy.

#### Read dataset

In [ ]:
df = spark.read.format("csv")\
  .option("header", True)\
  .option("inferSchema", True)\
  .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/company_bankruptcy_prediction_data.csv")
# print dataset size
print("records read: " + str(df.count()))
print("Schema: ")
df.printSchema()

In [ ]:
display(df)

#### Split the dataset into train and test

In [ ]:
train, test = df.randomSplit([0.85, 0.15], seed=1)

#### Add featurizer to convert features to vector

In [ ]:
from pyspark.ml.feature import VectorAssembler
feature_cols = df.columns[1:]
featurizer = VectorAssembler(
    inputCols=feature_cols,
    outputCol='features'
)
train_data = featurizer.transform(train)['Bankrupt?', 'features']
test_data = featurizer.transform(test)['Bankrupt?', 'features']

In [ ]:
train_data.show(10)

#### Check if the data is unbalanced

In [ ]:
train_data.groupBy("Bankrupt?").count().show()

#### Model Training

In [ ]:
from mmlspark.lightgbm import LightGBMClassifier
model = LightGBMClassifier(objective="binary", featuresCol="features", labelCol="Bankrupt?", isUnbalance=True)

In [ ]:
model = model.fit(train_data)

In [ ]:
from mmlspark.lightgbm import LightGBMClassificationModel
model.saveNativeModel("/lgbmcmodel")
model = LightGBMClassificationModel.loadNativeModelFromFile("/lgbmcmodel")

#### Feature Importances Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

feature_importances = model.getFeatureImportances()
fi = pd.Series(feature_importances,index = feature_cols)
fi = fi.sort_values(ascending = True)
f_index = fi.index
f_values = fi.values
 
# print feature importances 
print ('f_index:',f_index)
print ('f_values:',f_values)

# plo
x_index = list(range(len(fi)))
x_index = [x/len(fi) for x in x_index]
plt.rcParams['figure.figsize'] = (20,20)
plt.barh(x_index,f_values,height = 0.028 ,align="center",color = 'tan',tick_label=f_index)
plt.xlabel('importances')
plt.ylabel('features')
plt.show()

#### Model Prediction

In [ ]:
predictions = model.transform(test_data)
predictions.limit(10).toPandas()

In [ ]:
from mmlspark.train import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric="classification", labelCol='Bankrupt?', scoredLabelsCol='prediction').transform(predictions)
display(metrics)

## Quantile Regression for Drug Discovery with LightGBMRegressor

<img src="https://mmlspark.blob.core.windows.net/graphics/Documentation/drug.png" width="800" style="float: center;"/>

In this example, we show how to use LightGBM to build a simple regression model.

#### Read dataset

In [ ]:
triazines = spark.read.format("libsvm")\
    .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/triazines.scale.svmlight")

In [ ]:
# print some basic info
print("records read: " + str(triazines.count()))
print("Schema: ")
triazines.printSchema()
display(triazines.limit(10))

#### Split dataset into train and test

In [ ]:
train, test = triazines.randomSplit([0.85, 0.15], seed=1)

#### Model Training

In [ ]:
from mmlspark.lightgbm import LightGBMRegressor
model = LightGBMRegressor(objective='quantile',
                          alpha=0.2,
                          learningRate=0.3,
                          numLeaves=31).fit(train)

In [ ]:
print(model.getFeatureImportances())

#### Model Prediction

In [ ]:
scoredData = model.transform(test)
display(scoredData)

In [ ]:
from mmlspark.train import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric='regression',
                                 labelCol='label',
                                 scoresCol='prediction') \
            .transform(scoredData)
display(metrics)

## LightGBM Ranker

#### Read dataset

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import explode

In [ ]:
df1 = spark.read.format('libsvm') \
    .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/lightGBMRanker_rank_train.libsvm") \
    .withColumn('iid', monotonically_increasing_id())
display(df1)

In [ ]:
import numpy as np
from pyspark.sql.types import ArrayType, LongType
def create_rows(value, index):
    arr = np.zeros([value])
    arr.fill(index)
    return arr.astype('int').tolist()
create_rows_udf = udf(create_rows, ArrayType(LongType()))

In [ ]:
query_col = 'query'
label_col = 'labels'
# read in CSV file
df2 = spark.read.format('csv').option('inferSchema', True) \
    .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/lightGBMRanker_rank_train_query.csv") \
    .withColumn('index', monotonically_increasing_id()) \
    .withColumn(query_col, explode(create_rows_udf('_c0', 'index'))) \
    .withColumn('iid', monotonically_increasing_id()) \
    .drop('_c0', 'index') \
    .join(df1, 'iid').drop('iid') \
    .withColumnRenamed('label', label_col)

In [ ]:
# print some basic info
print("records read: " + str(df1.count()))
print("Schema: ")
df2.printSchema()
display(df2.limit(10))

#### Model Training

In [ ]:
from mmlspark.lightgbm import LightGBMRanker

features_col = 'features'
lgbm_ranker = LightGBMRanker(labelCol=label_col,
                             featuresCol=features_col,
                             groupCol=query_col,
                             predictionCol='preds',
                             leafPredictionCol='leafPreds',
                             featuresShapCol='importances',
                             repartitionByGroupingColumn=True,
                             numLeaves=32,
                             numIterations=200,
                             evalAt=[1,3,5],
                             metric='ndcg')

In [ ]:
lgbm_ranker_model = lgbm_ranker.fit(df2)

#### Model Prediction

In [ ]:
dt1 = spark.read.format('libsvm') \
    .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/lightGBMRanker_rank_test.libsvm") \
    .withColumn('iid', monotonically_increasing_id())
dt2 = spark.read.format('csv').option('inferSchema', True) \
    .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/lightGBMRanker_rank_test_query.csv") \
    .withColumn('index', monotonically_increasing_id()) \
    .withColumn(query_col, explode(create_rows_udf('_c0', 'index'))) \
    .withColumn('iid', monotonically_increasing_id()) \
    .drop('_c0', 'index') \
    .join(dt1, 'iid').drop('iid') \
    .withColumnRenamed('label', label_col)

In [ ]:
predictions = lgbm_ranker_model.transform(dt2)
predictions.limit(10).toPandas()